### 1. Download the page and start parsing it
    - Download the webpage containg the forecast
    - Create a `BeautifulSoup` class to parse the page 
    - Find the `div` with id `seven-day-forecast`, and assign to `seven_day`
    - Inside `seven-day`, find each individual forecast item 
    - Extract and print the first forecast item 
    

In [16]:
# import the necessary libraries 
import requests 
from bs4 import BeautifulSoup 

# Download the page using GET request
page = requests.get("https://forecast.weather.gov/MapClick.php?lat=37.7772&lon=-122.4168")

# Create a BeautifulSoup class to parse the page
soup = BeautifulSoup(page.content, 'html.parser')

# Find the `div` with id `seven-day-forecast`, and assign to `seven_day`
seven_day = soup.find(id = "seven-day-forecast")
#print(seven_day)

#Inside `seven-day`, find each individual forecast item 
forecast_items = seven_day.find_all(class_="tombstone-container")
#Inside `seven-day`, find each individual forecast item
tonight = forecast_items[0]
# Extract and print 
print(tonight.prettify())

<div class="tombstone-container">
 <p class="period-name">
  Overnight
  <br/>
  <br/>
 </p>
 <p>
  <img alt="Overnight: Mostly cloudy, with a low around 48. North northwest wind around 7 mph. " class="forecast-icon" src="newimages/medium/nbkn.png" title="Overnight: Mostly cloudy, with a low around 48. North northwest wind around 7 mph. "/>
 </p>
 <p class="short-desc">
  Mostly Cloudy
 </p>
 <p class="temp temp-low">
  Low: 48 °F
 </p>
</div>


### 2. Extracting information from the page
As we can see, inside the forecast item `Overnight` is all the information we want. There are four pieces of information we can extract:

    -The name of the forecast item — in this case, `Overnight`.
    -The description of the conditions — this is stored in the title property of img.
    -A short description of the conditions — in this case, Mostly Cloudy.
    -The temperature low — in this case, 48 degrees.

In [19]:
#Extract the name of forecast item which is Overnight
period = tonight.find(class_="period-name").get_text()

#The description of the conditions — this is stored in the title property of img.
img_title = tonight.find("img")['title']

# Extract the description of the conditions — this is stored in the title property of img.
short_desc = tonight.find(class_="short-desc").get_text()

# Extract the temperature 
temp = tonight.find(class_="temp temp-low").get_text()

# 

# Print the information
print(period)
print(img_title)
print(short_desc)
print(temp)

Overnight
Overnight: Mostly cloudy, with a low around 48. North northwest wind around 7 mph. 
Mostly Cloudy
Low: 48 °F


### 3. Extracting all the information from the page 
Now that we know how to extract each individual piece of information, we can combine our knowledge with CSS selectors and list comprehensions to **extract everything at once**.
In the below code, we will:

    -Select all items with the class `period-name` inside an item with the class `tombstone-container` in `seven_day`.
    -Use a list comprehension to call the `get_text` method on each `BeautifulSoup` object.

In [21]:
#Select all items with the class `period-name` inside an item with the class `tombstone-container` in `seven_day
period_tags = seven_day.select(".tombstone-container .period-name")
periods = [pt.get_text() for pt in period_tags]
periods

['Overnight',
 'Monday',
 'MondayNight',
 'Tuesday',
 'TuesdayNight',
 'Wednesday',
 'WednesdayNight',
 'Thursday',
 'ThursdayNight']

In [22]:
# Use a list comprehension to call the `get_text` method on each `BeautifulSoup` object.
short_descs = [sd.get_text() for sd in seven_day.select(".tombstone-container .short-desc")]
temps = [t.get_text() for t in seven_day.select(".tombstone-container .temp")]
descs = [d["title"] for d in seven_day.select(".tombstone-container img")]

print(short_descs)
print(temps)
print(descs)

['Mostly Cloudy', 'Cloudy thenRain', 'Showers andBreezy', 'Showers andBreezy thenRain Likely', 'Chance Rain', 'Chance Rain', 'Rain', 'Rain', 'Rain']
['Low: 48 °F', 'High: 59 °F', 'Low: 55 °F', 'High: 60 °F', 'Low: 48 °F', 'High: 57 °F', 'Low: 49 °F', 'High: 57 °F', 'Low: 52 °F']
['Overnight: Mostly cloudy, with a low around 48. North northwest wind around 7 mph. ', 'Monday: Rain after 5pm.  High near 59. North northeast wind around 5 mph becoming calm.  Chance of precipitation is 80%.', 'Monday Night: Rain before 2am, then showers and possibly a thunderstorm after 2am.  Steady temperature around 55. Breezy, with a south wind 7 to 12 mph increasing to 21 to 26 mph after midnight. Winds could gust as high as 43 mph.  Chance of precipitation is 100%. New rainfall amounts between a half and three quarters of an inch possible. ', 'Tuesday: Rain and possibly a thunderstorm before 11am, then rain likely.  High near 60. Breezy, with a southwest wind 15 to 23 mph, with gusts as high as 39 mph. 

### 4. Combining our data into a Pandas Dataframe


In [23]:
# Import pandas library
import pandas as pd 

weather = pd.DataFrame({
    "period": periods,
    "short_desc": short_descs,
    "temp": temps,
    "desc": descs
})

weather

,period,short_desc,temp,desc
0,Overnight,Mostly Cloudy,Low: 48 °F,"Overnight: Mostly cloudy, with a low around 48..."
1,Monday,Cloudy thenRain,High: 59 °F,Monday: Rain after 5pm. High near 59. North n...
2,MondayNight,Showers andBreezy,Low: 55 °F,"Monday Night: Rain before 2am, then showers an..."
3,Tuesday,Showers andBreezy thenRain Likely,High: 60 °F,Tuesday: Rain and possibly a thunderstorm befo...
4,TuesdayNight,Chance Rain,Low: 48 °F,"Tuesday Night: A 40 percent chance of rain, ma..."
5,Wednesday,Chance Rain,High: 57 °F,"Wednesday: A 30 percent chance of rain, mainly..."
6,WednesdayNight,Rain,Low: 49 °F,"Wednesday Night: Rain, mainly after 11pm. Low..."
7,Thursday,Rain,High: 57 °F,"Thursday: Rain. Mostly cloudy, with a high ne..."
8,ThursdayNight,Rain,Low: 52 °F,"Thursday Night: Rain. Cloudy, with a low arou..."
